In [11]:
!pip install networkx

In [12]:
import torch
import torch.nn.functional as F
import networkx as nx
import numpy as np
import torch.optim as optim
from itertools import combinations
import pickle

In [13]:
# Define a graph
# n = 50  # number of nodes
# m = 613  # number of edges
# G = nx.gnm_random_graph(n, m, seed = 0)

#G = nx.gnm_random_graph(n, m, seed = 4) # This graph has a MIS of 5, and it is hard to find!!! They are {0, 2, 9, 10, 11} and {0, 2, 8, 9, 11}
# Create a G(n, m) random graph
def destringizer(value):
    try:
        # Try to convert the string to an integer
        return int(value)
    except ValueError:
        try:
            # Try to convert the string to a float
            return float(value)
        except ValueError:
            # If conversion fails, return the original string
            return value

# Load the .gml graph using nx.read_gml and provide the destringizer function
#G = nx.read_gml("GNM_n50_m613_seed9.gml", destringizer=destringizer)
#G = nx.read_gml("/content/GNM_n500_m62375_seed0.gml", destringizer=destringizer)

#G = nx.read_gpickle("/content/ER_700_800_0.15_0.gpickle")

file_path = "./graphs/er_700-800/ER_700_800_0.15_12.gpickle"

with open(file_path, 'rb') as f:
    G = pickle.load(f)

    G = nx.relabel.convert_node_labels_to_integers(G, first_label=0)

n = len(G.nodes)
m = len(G.edges)
#print(G.edges)
complement_G = nx.complement(G)

print(n,m)



### Visualize the graph
#nx.draw(G, with_labels=True, font_weight='bold')

705 37218


In [14]:
### Obtain the A_G matrix

adjacency_matrix = nx.adjacency_matrix(G)
adjacency_matrix_dense = adjacency_matrix.todense()
adjacency_matrix_tensor = torch.tensor(adjacency_matrix_dense, dtype=torch.float32)

### Obtain the A_G_hat matrix

adjacency_matrix_comp = nx.adjacency_matrix(complement_G)
adjacency_matrix_dense_comp = adjacency_matrix_comp.todense()
adjacency_matrix_tensor_comp = torch.tensor(adjacency_matrix_dense_comp, dtype=torch.float32)

def loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma, beta):
    ## without edges of the comp graph:
    # loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X)

    ## with edges of the comp graph:
    loss = -Matrix_X.sum() + (gamma/2) * (Matrix_X.T @ (adjacency_matrix_tensor) @ Matrix_X) - (beta/2) * (Matrix_X.T @ (adjacency_matrix_tensor_comp) @ Matrix_X)
    return loss

# # test function:
# Matrix_X = torch.rand((len(G.nodes),1), requires_grad=True)
# print("testing the function: ", loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma=5, beta = 1))


In [15]:
# Constructing the constant Hessian and looking at the eigrn values: This depends on the connectivity of graph... The more edges, the worst, which we already know
gamma = 45 # I select this from the best solution from ReduMIS: Double check the bounds with Rongrong
beta = 1
Hessian = torch.zeros(n, n)
complement_G = nx.complement(G)

Hessian_with_third = (gamma/2)*adjacency_matrix_tensor - (beta/2)*adjacency_matrix_tensor_comp
Hessain_without    = (gamma/2)*adjacency_matrix_tensor

# Compute eigenvalues
eigenvalues_with, _ = torch.linalg.eig(Hessian_with_third)

# Extract the real part of the eigenvalues
real_eigenvalues_with = eigenvalues_with.real


# Compute eigenvalues
eigenvalues_without, _ = torch.linalg.eig(Hessain_without)

# Extract the real part of the eigenvalues
real_eigenvalues_without = eigenvalues_without.real

#print("Eigenvalues without:", real_eigenvalues_without)

# Count positive values
positive_count_with = torch.sum(real_eigenvalues_with > 0).item()

# Count negative values
negative_count_with = torch.sum(real_eigenvalues_with < 0).item()

print("Number of positive values with:", positive_count_with)
print("Number of negative values with:", negative_count_with)

# Count positive values
positive_count_without = torch.sum(real_eigenvalues_without > 0).item()

# Count negative values
negative_count_without = torch.sum(real_eigenvalues_without < 0).item()

print("Number of positive values without:", positive_count_without)
print("Number of negative values without:", negative_count_without)


Number of positive values with: 350
Number of negative values with: 355
Number of positive values without: 349
Number of negative values without: 356


In [23]:
# Optimization loop:
import time
# Initialization:
torch.manual_seed(115)

device = torch.device(
    "cuda:0" if torch.cuda.is_available() else "cpu"
)
print("using device: ", device)

Matrix_X = torch.ones((n), requires_grad=True, device=device)
# Matrix_X = torch.rand((n), requires_grad=True, device=device)
X_ini  = Matrix_X.data.clone()

# dict for saving... I am thinking of diversification by looking at previous initializations... Still under investigation
dict_of_inits = {}

# This is obtained to get a sense of how far are we from the init
#gamma = torch.tensor(50.0, requires_grad=True)

learning_rate_alpha = 0.5
number_of_iterations_T = 75000

adjacency_matrix_tensor = adjacency_matrix_tensor.to(device)
adjacency_matrix_tensor_comp = adjacency_matrix_tensor_comp.to(device)

# Define Optimizer over matrix X
optimizer = optim.Adam([Matrix_X], lr=learning_rate_alpha)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1)

Best_MIS = 0
MIS = []
test_runtime = False

improve_iteration = 0

for iteration_t in range(number_of_iterations_T):
    
    start_time = time.time()

    loss = loss_function(adjacency_matrix_tensor,adjacency_matrix_tensor_comp, Matrix_X, gamma = 950, beta = 1)

    if test_runtime:
      torch.cuda.synchronize()
      loss_time = time.time()
      print("time to compute loss function:", loss_time - start_time)

    optimizer.zero_grad()  # Clear gradients for the next iteration

    if test_runtime:
      torch.cuda.synchronize()
      zero_grad_time = time.time()
      print("time to zero gradients:", zero_grad_time - loss_time)

    loss.backward()  # Backpropagation

    if test_runtime:
      torch.cuda.synchronize()
      backpropagation_time = time.time()
      print("time to compute back propagation:", backpropagation_time - zero_grad_time)

    optimizer.step()  # Update the parameters

    if test_runtime:
      torch.cuda.synchronize()
      optim_step_time = time.time()
      print("time to compute step time:", optim_step_time - backpropagation_time)

    # Box-constraining:
    Matrix_X.data[Matrix_X>=1] =1
    Matrix_X.data[Matrix_X<=0] =0

    b = Matrix_X.data > 0.05
    indices = b.nonzero(as_tuple=True)[0].tolist()

    subgraph = G.subgraph(indices)

    if test_runtime:
      torch.cuda.synchronize()
      box_constraint_time = time.time()
      print("time to perform box constraining:", box_constraint_time - optim_step_time)


    # Report the current MIS
    # MIS = []
    # for node in G.nodes:
    #     if Matrix_X[node] >0.0:
    #       MIS.append(node)
    IS = indices

    
    if test_runtime:
      torch.cuda.synchronize()
      MIS_generation_time = time.time()
      print("time to perform MIS selection:", MIS_generation_time - box_constraint_time)

    # If no MIS, move one
    # if MIS_checker(MIS, G)[0] is False: MIS = []
    if any(subgraph.edges()): IS = []

    if test_runtime:
      torch.cuda.synchronize()
      MIS_check_time = time.time()
      print("time to perform MIS checking:", MIS_check_time - MIS_generation_time)

    # Iteration logger every XX iterations:
    if (iteration_t + 1) % 200 == 0:
        print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, MIS Size: {Best_MIS}")
        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}, grad norm: {torch.norm(Matrix_X.grad).item()}")
        #print(f"Step {iteration_t + 1}/{number_of_iterations_T}, IS: {MIS}, lr: {learning_rate_alpha}, Loss: {loss.item()}")
    if len(IS) > 0:
      if improve_iteration == 0:
        lr_scheduler.step()
      improve_iteration += 1

      if len(IS) > Best_MIS:
        Best_MIS = len(IS)
        MIS = IS

      # print("+++++++++++ A MIS is found with size: ", [[[len(IS)]]], "++++ BEST so far",[Best_MIS], "Dist from Init: =",l2_norm.item())

      # # Restart X and the optimizer to search at a different point in [0,1]^n
      if (improve_iteration > 50):
        improve_iteration = 0

        Matrix_X = torch.nn.init.uniform_(torch.empty((n), requires_grad=True, device=device))
        optimizer = optim.AdamW([Matrix_X], lr=learning_rate_alpha)
        lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=0.5)



# # Visualize the solution:
# node_colors = ['green' if node in MIS else 'blue' for node in G.nodes]
# nx.draw(G, with_labels=True, font_weight='bold', node_color=node_colors)

using device:  cuda:0
Step 200/40000, IS: [], lr: 0.5, MIS Size: 0
Step 400/40000, IS: tensor([  4,  36,  86,  91, 105, 123, 137, 143, 153, 200, 202, 206, 215, 218,
        225, 226, 227, 230, 235, 283, 286, 318, 370, 421, 438, 455, 467, 474,
        488, 506, 583, 626, 672, 674, 688, 704], device='cuda:0'), lr: 0.5, MIS Size: 36
Step 600/40000, IS: tensor([  4,  36,  86,  91, 105, 123, 137, 143, 153, 200, 202, 206, 215, 218,
        225, 226, 227, 230, 235, 283, 286, 318, 370, 421, 438, 455, 467, 474,
        488, 506, 583, 626, 672, 674, 688, 704], device='cuda:0'), lr: 0.5, MIS Size: 36
Step 800/40000, IS: tensor([  4,  36,  86,  91, 105, 123, 137, 143, 153, 200, 202, 206, 215, 218,
        225, 226, 227, 230, 235, 283, 286, 318, 370, 421, 438, 455, 467, 474,
        488, 506, 583, 626, 672, 674, 688, 704], device='cuda:0'), lr: 0.5, MIS Size: 36
Step 1000/40000, IS: tensor([  4,  36,  86,  91, 105, 123, 137, 143, 153, 200, 202, 206, 215, 218,
        225, 226, 227, 230, 235, 283, 2

KeyboardInterrupt: 

In [ ]:
print(len(dict_of_inits))

from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

#init_points_torch = torch.zeros(len(dict_of_inits), n)
init_points_torch = torch.stack(list(dict_of_inits.keys()))
init_points_torch_numpy = init_points_torch.numpy()

print(init_points_torch.size())

pca = PCA(n_components=2)

# Fit the PCA model and transform the data
pca_result = pca.fit_transform(init_points_torch_numpy)

# Visualize the result
plt.scatter(pca_result[:, 0], pca_result[:, 1])
plt.title('PCA Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

1


ModuleNotFoundError: No module named 'sklearn'